In [ ]:
testing with video feed

In [ ]:
import cv2
import numpy as np
import os
from plyer import notification

# Load YOLOv4 model and class labels
net = cv2.dnn.readNetFromDarknet('yolov4_custom_1.cfg', 'yolov4_custom_12000.weights')
classes = ['Cycling specific sign', 'Directional sign', 'Hazard sign', 'Stop sign', 'Warning sign', 'Pedestrian sign', 'Zebra Crossing']

# Create video capture object
video_path = r'C:\Users\Bhoyee-pc\Downloads\pro-sch\sample_data\videos\video5.mp4'
cap = cv2.VideoCapture(video_path)

# Define the codec and create VideoWriter object
output_path = r'C:\Users\Bhoyee-pc\Downloads\pro-sch\sample_data\videos\output.mp4'
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (1280, 720))

notification_triggered = False

while cap.isOpened():
    # Read frame from the video feed
    ret, frame = cap.read()

    if not ret:
        break

    # Resize frame
    frame = cv2.resize(frame, (1280, 720))
    height, width, _ = frame.shape

    # Preprocess frame for object detection
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(net.getUnconnectedOutLayersNames())

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, color, 2)
            
            # Display notification when sign is detected
            if not notification_triggered and x > 5 and y > 5:
                notification_title = "Sign Detected"
                notification_text = f"A {label} sign was detected."
                notification.notify(title=notification_title, message=notification_text, timeout=5)
                notification_triggered = True

    out.write(frame)

# Release video capture object, VideoWriter object, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
Testing with camera

In [ ]:
import cv2
import numpy as np
import os
from plyer import notification

# Load YOLOv4 model and class labels
net = cv2.dnn.readNetFromDarknet('yolov4_custom_1.cfg', 'yolov4_custom_12000.weights')
classes = ['Cycling specific sign', 'Directional sign', 'Hazard sign', 'Stop sign', 'Warning sign', 'Pedestrian sign', 'Zebra Crossing']

# Create video capture object
cap = cv2.VideoCapture(0)  # Use index 0 for the default camera

# Define the codec and create VideoWriter object
output_path = os.path.join('sample_data', 'videos', 'output.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (1280, 720))

while cap.isOpened():
    # Read frame from the video feed
    ret, frame = cap.read()

    if not ret:
        break

    # Resize frame
    frame = cv2.resize(frame, (1280, 720))
    height, width, _ = frame.shape

    # Preprocess frame for object detection
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(net.getUnconnectedOutLayersNames())

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))

    # Display bounding boxes and labels
    detected_signs = []  # List to store the detected signs
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, color, 2)

            # Add the detected sign to the list
            detected_signs.append(label)

    # Display notification when road signs are detected
    if detected_signs:
        notification_title = "Sign Detected"
        notification_text = f"A {', '.join(detected_signs)} sign(s) was detected."
        notification.notify(title=notification_title, message=notification_text, timeout=5)

    out.write(frame)

# Release video capture object, VideoWriter object, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
showling live camera

In [ ]:
import cv2
import numpy as np
import os
from plyer import notification

# Load YOLOv4 model and class labels
net = cv2.dnn.readNetFromDarknet('yolov4_custom_1.cfg', 'yolov4_custom_12000.weights')
classes = ['Cycling specific sign', 'Directional sign', 'Hazard sign', 'Stop sign', 'Warning sign', 'Pedestrian sign', 'Zebra Crossing']

# Create video capture object
cap = cv2.VideoCapture(0)  # Use index 0 for the default camera

# Define the codec and create VideoWriter object
output_path = 'output.mp4'
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (1280, 720))

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Resize frame
    frame = cv2.resize(frame, (1280, 720))
    height, width, _ = frame.shape

    # Preprocess frame for object detection
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(net.getUnconnectedOutLayersNames())

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))

    # Display bounding boxes and labels
    detected_signs = []  # List to store the detected signs
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, color, 2)

            # Add the detected sign to the list
            detected_signs.append(label)

    # Display notification when road signs are detected
    if detected_signs:
        notification_title = "Sign Detected"
        notification_text = f"A {', '.join(detected_signs)} sign(s) was detected."
        notification.notify(title=notification_title, message=notification_text, timeout=5)

    cv2.imshow("Object Detection", frame)
    out.write(frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release video capture object, VideoWriter object, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
testing voice with life camera

In [ ]:
import cv2
import numpy as np
from IPython.display import display
import pyttsx3

# Load YOLOv4 model and class labels
net = cv2.dnn.readNetFromDarknet('yolov4_custom_1.cfg', 'yolov4_custom_12000.weights')
classes = ['Cycling specific sign', 'Directional sign', 'Hazard sign', 'Stop sign', 'Warning sign', 'Pedestrian sign', 'Zebra Crossing']

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Function to perform object detection on each frame
"""
This function performs object detection on each frame of the video stream. 
The frame is preprocessed for object detection, and the YOLOv4 model is used to detect objects.
The bounding boxes and labels are drawn on the frame, and the detected signs are stored in a list. 
If signs are detected, a notification text is created, and the text is spoken using the text-to-speech engine.
"""
def detect_objects(frame):
    height, width, _ = frame.shape

    # Preprocess frame for object detection
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(net.getUnconnectedOutLayersNames())

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))

    # Display bounding boxes and labels
    detected_signs = []  # List to store the detected signs
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, color, 2)

            # Add the detected sign to the list
            detected_signs.append(label)

    # Speak the notification when road signs are detected
    if detected_signs:
        notification_text = f"A {', '.join(detected_signs)} sign(s) was detected."
        engine.say(notification_text)
        engine.runAndWait()

    return frame

# Function to handle the video stream
"""
This function handles the video stream. It captures video frames from the PC camera using cv2.VideoCapture
with index 0. You can change the index if you have multiple cameras connected. 
The function reads frames from the camera, calls the detect_objects function to perform object detection, 
displays the processed frames, and exits when the 'q' key is pressed.
"""
def handle_video_stream():
    cap = cv2.VideoCapture(0)  # Open the PC camera (change the index if multiple cameras are available)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_objects(frame)
        cv2.imshow('Video Stream', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the function to start video streaming and object detection
handle_video_stream()
